In [ ]:
!pip install flwr["simulation"]==0.18.0 torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 106 kB 14.8 MB/s 
     |████████████████████████████████| 4.1 MB 40.6 MB/s 
     |████████████████████████████████| 54.5 MB 152 kB/s 
     |████████████████████████████████| 8.8 MB 46.4 MB/s 
     |████████████████████████████████| 144 kB 46.9 MB/s 
     |████████████████████████████████| 57 kB 5.6 MB/s 
     |████████████████████████████████| 201 kB 60.4 MB/s 
     |████████████████████████████████| 1.1 MB 38.5 MB/s 
     |████████████████████████████████| 128 kB 59.9 MB/s 
     |████████████████████████████████| 3.1 MB 47.3 MB/s 
     |████████████████████████████████| 82 kB 270 kB/s 
     |████████████████████████████████| 94 kB 3.9 MB/s 
     |████████████████████████████████| 271 kB 76.1 MB/s 
     |████████████████████████████████| 58 kB 6.5 MB/s 
     |████████████████████████████████| 461 kB 47.1 MB/s 
  Created wheel for gpustat: filename=g

In [ ]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import flwr as fl
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

DEVICE = torch.device("cpu")
# DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")

Training on cpu


load the CIFAR-10 training and test set, partition them into ten smaller datasets

In [ ]:
NUM_CLIENTS = 10

def load_datasets(num_clients: int):
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
      [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
    testset = CIFAR10("./dataset", train=False, download=True, transform=transform)

    # Split training set into `num_clients` partitions to simulate different local datasets
    partition_size = len(trainset) // num_clients
    lengths = [partition_size] * num_clients
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=32, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=32))
    testloader = DataLoader(testset, batch_size=32)
    return trainloaders, valloaders, testloader

trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./dataset/cifar-10-python.tar.gz to ./dataset
Files already downloaded and verified


Model training/evaluation

In [ ]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(testloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

Implement Flower client

In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

Server-side parameter initialization

Flower, by default, initializes the global model by asking one random client for the initial parameters.

In [ ]:
# Create an instance of the model and get the parameters
params = get_parameters(Net())

# Pass parameters to the Strategy for server-side parameter initialization
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_eval=0.3,
    min_fit_clients=3,
    min_eval_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.weights_to_parameters(params),
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    num_rounds=3,  # Just three rounds
    strategy=strategy,
)

INFO flower 2022-06-23 14:17:06,934 | app.py:147 | Ray initialized with resources: {'CPU': 2.0, 'memory': 7887246951.0, 'node:172.28.0.2': 1.0, 'object_store_memory': 3943623475.0}
INFO flower 2022-06-23 14:17:06,944 | app.py:156 | Starting Flower simulation running: {'num_rounds': 3}
INFO flower 2022-06-23 14:17:06,948 | server.py:128 | Initializing global parameters
INFO flower 2022-06-23 14:17:06,949 | server.py:323 | Using initial parameters provided by strategy
INFO flower 2022-06-23 14:17:06,952 | server.py:130 | Evaluating initial parameters
INFO flower 2022-06-23 14:17:06,957 | server.py:143 | FL starting
DEBUG flower 2022-06-23 14:17:06,959 | server.py:269 | fit_round: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=274) [Client 4] fit, config: {}
(launch_and_fit pid=273) [Client 1] fit, config: {}
(launch_and_fit pid=274) Epoch 1: train loss 0.028955044224858284, accuracy 0.23066666666666666
(launch_and_fit pid=273) Epoch 1: train loss 0.02939736284315586, accuracy 0.22355555555555556
(launch_and_fit pid=274) [Client 2] fit, config: {}


DEBUG flower 2022-06-23 14:17:21,447 | server.py:281 | fit_round received 3 results and 0 failures
DEBUG flower 2022-06-23 14:17:21,466 | server.py:215 | evaluate_round: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=274) Epoch 1: train loss 0.029448186978697777, accuracy 0.216
(launch_and_evaluate pid=274) [Client 2] evaluate, config: {}
(launch_and_evaluate pid=273) [Client 4] evaluate, config: {}


DEBUG flower 2022-06-23 14:17:26,740 | server.py:227 | evaluate_round received 3 results and 0 failures
DEBUG flower 2022-06-23 14:17:26,742 | server.py:269 | fit_round: strategy sampled 3 clients (out of 10)


...

DEBUG flower 2022-06-23 14:17:54,878 | server.py:281 | fit_round received 3 results and 0 failures
DEBUG flower 2022-06-23 14:17:54,894 | server.py:215 | evaluate_round: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=273) Epoch 1: train loss 0.024053696542978287, accuracy 0.38555555555555554
(launch_and_evaluate pid=274) [Client 7] evaluate, config: {}
(launch_and_evaluate pid=273) [Client 4] evaluate, config: {}


DEBUG flower 2022-06-23 14:18:00,050 | server.py:227 | evaluate_round received 3 results and 0 failures
INFO flower 2022-06-23 14:18:00,052 | server.py:182 | FL finished in 53.09333215900003
INFO flower 2022-06-23 14:18:00,058 | app.py:149 | app_fit: losses_distributed [(1, 0.06236810111999511), (2, 0.05608209506670634), (3, 0.05276714571317037)]
INFO flower 2022-06-23 14:18:00,061 | app.py:150 | app_fit: metrics_distributed {}
INFO flower 2022-06-23 14:18:00,063 | app.py:151 | app_fit: losses_centralized []
INFO flower 2022-06-23 14:18:00,065 | app.py:152 | app_fit: metrics_centralized {}


(launch_and_evaluate pid=274) [Client 0] evaluate, config: {}


History (loss, distributed):
	round 1: 0.06236810111999511
	round 2: 0.05608209506670634
	round 3: 0.05276714571317037

Starting with a customized strategy

The strategy encapsulates the federated learning approach/algorithm, for example, FedAvg or FedAdagrad.So Now try with both strategy

In [ ]:
# Create FedAdam strategy
strategy=fl.server.strategy.FedAdagrad(
    fraction_fit=0.3,
    fraction_eval=0.3,
    min_fit_clients=3,
    min_eval_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.weights_to_parameters(get_parameters(Net())),
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    num_rounds=3,  # Just three rounds
    strategy=strategy,
)

INFO flower 2022-06-23 14:18:06,993 | app.py:147 | Ray initialized with resources: {'node:172.28.0.2': 1.0, 'memory': 7890041243.0, 'CPU': 2.0, 'object_store_memory': 3945020620.0}
INFO flower 2022-06-23 14:18:06,997 | app.py:156 | Starting Flower simulation running: {'num_rounds': 3}
INFO flower 2022-06-23 14:18:07,001 | server.py:128 | Initializing global parameters
INFO flower 2022-06-23 14:18:07,004 | server.py:323 | Using initial parameters provided by strategy
INFO flower 2022-06-23 14:18:07,007 | server.py:130 | Evaluating initial parameters
INFO flower 2022-06-23 14:18:07,010 | server.py:143 | FL starting
DEBUG flower 2022-06-23 14:18:07,014 | server.py:269 | fit_round: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=534) [Client 4] fit, config: {}
(launch_and_fit pid=535) [Client 9] fit, config: {}
(launch_and_fit pid=534) Epoch 1: train loss 0.028966635465621948, accuracy 0.234
(launch_and_fit pid=535) Epoch 1: train loss 0.029385993257164955, accuracy 0.228
(launch_and_fit pid=534) [Client 3] fit, config: {}


DEBUG flower 2022-06-23 14:18:19,624 | server.py:281 | fit_round received 3 results and 0 failures
DEBUG flower 2022-06-23 14:18:19,649 | server.py:215 | evaluate_round: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=534) Epoch 1: train loss 0.029441609978675842, accuracy 0.21733333333333332
(launch_and_evaluate pid=534) [Client 5] evaluate, config: {}
(launch_and_evaluate pid=535) [Client 7] evaluate, config: {}


DEBUG flower 2022-06-23 14:18:24,792 | server.py:227 | evaluate_round received 3 results and 0 failures
DEBUG flower 2022-06-23 14:18:24,794 | server.py:269 | fit_round: strategy sampled 3 clients (out of 10)


...

DEBUG flower 2022-06-23 14:18:51,652 | server.py:281 | fit_round received 3 results and 0 failures
DEBUG flower 2022-06-23 14:18:51,671 | server.py:215 | evaluate_round: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=535) Epoch 1: train loss 0.038299087435007095, accuracy 0.12866666666666668
(launch_and_evaluate pid=535) [Client 8] evaluate, config: {}
(launch_and_evaluate pid=534) [Client 6] evaluate, config: {}


DEBUG flower 2022-06-23 14:18:56,700 | server.py:227 | evaluate_round received 3 results and 0 failures
INFO flower 2022-06-23 14:18:56,703 | server.py:182 | FL finished in 49.68908798199999
INFO flower 2022-06-23 14:18:56,710 | app.py:149 | app_fit: losses_distributed [(1, 3.3032486699422203), (2, 0.35022931861877443), (3, 0.09931291103363038)]
INFO flower 2022-06-23 14:18:56,712 | app.py:150 | app_fit: metrics_distributed {}
INFO flower 2022-06-23 14:18:56,718 | app.py:151 | app_fit: losses_centralized []
INFO flower 2022-06-23 14:18:56,720 | app.py:152 | app_fit: metrics_centralized {}


(launch_and_evaluate pid=535) [Client 1] evaluate, config: {}


History (loss, distributed):
	round 1: 3.3032486699422203
	round 2: 0.35022931861877443
	round 3: 0.09931291103363038

In [ ]:
# The `evaluate` function will be by Flower called after every round
def evaluate(
    weights: fl.common.Weights,
) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    net = Net()
    valloader = valloaders[0]
    set_parameters(net, weights)  # Update model with the latest parameters
    loss, accuracy = test(net, valloader)
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy}

In [ ]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_eval=0.3,
    min_fit_clients=3,
    min_eval_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.weights_to_parameters(get_parameters(Net())),
    eval_fn=evaluate,  # Pass the evaluation function
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    num_rounds=3,  # Just three rounds
    strategy=strategy,
)

INFO flower 2022-06-23 14:19:02,939 | app.py:147 | Ray initialized with resources: {'object_store_memory': 3945019392.0, 'memory': 7890038784.0, 'CPU': 2.0, 'node:172.28.0.2': 1.0}
INFO flower 2022-06-23 14:19:02,943 | app.py:156 | Starting Flower simulation running: {'num_rounds': 3}
INFO flower 2022-06-23 14:19:02,948 | server.py:128 | Initializing global parameters
INFO flower 2022-06-23 14:19:02,950 | server.py:323 | Using initial parameters provided by strategy
INFO flower 2022-06-23 14:19:02,955 | server.py:130 | Evaluating initial parameters
INFO flower 2022-06-23 14:19:03,267 | server.py:137 | initial parameters (loss, other metrics): 0.07375648212432862, {'accuracy': 0.092}
INFO flower 2022-06-23 14:19:03,274 | server.py:143 | FL starting
DEBUG flower 2022-06-23 14:19:03,280 | server.py:269 | fit_round: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.07375648212432862 / accuracy 0.092
(launch_and_fit pid=778) [Client 6] fit, config: {}
(launch_and_fit pid=777) [Client 9] fit, config: {}
(launch_and_fit pid=778) Epoch 1: train loss 0.029604105278849602, accuracy 0.20266666666666666
(launch_and_fit pid=777) Epoch 1: train loss 0.029293237254023552, accuracy 0.21555555555555556
(launch_and_fit pid=778) [Client 4] fit, config: {}


DEBUG flower 2022-06-23 14:19:15,889 | server.py:281 | fit_round received 3 results and 0 failures
INFO flower 2022-06-23 14:19:16,098 | server.py:164 | fit progress: (1, 0.06275381827354432, {'accuracy': 0.304}, 12.817744654999984)
INFO flower 2022-06-23 14:19:16,100 | server.py:209 | evaluate_round: no clients selected, cancel
DEBUG flower 2022-06-23 14:19:16,117 | server.py:269 | fit_round: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=778) Epoch 1: train loss 0.02880917303264141, accuracy 0.22866666666666666
Server-side evaluation loss 0.06275381827354432 / accuracy 0.304
(launch_and_fit pid=778) [Client 1] fit, config: {}
(launch_and_fit pid=777) [Client 7] fit, config: {}
(launch_and_fit pid=778) Epoch 1: train loss 0.025630448013544083, accuracy 0.3422222222222222
(launch_and_fit pid=777) Epoch 1: train loss 0.02544352412223816, accuracy 0.3437777777777778
(launch_and_fit pid=778) [Client 4] fit, config: {}


DEBUG flower 2022-06-23 14:19:27,425 | server.py:281 | fit_round received 3 results and 0 failures
INFO flower 2022-06-23 14:19:27,628 | server.py:164 | fit progress: (2, 0.05512853932380676, {'accuracy': 0.362}, 24.347483306999976)
INFO flower 2022-06-23 14:19:27,629 | server.py:209 | evaluate_round: no clients selected, cancel
DEBUG flower 2022-06-23 14:19:27,636 | server.py:269 | fit_round: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=778) Epoch 1: train loss 0.025804979726672173, accuracy 0.32911111111111113
Server-side evaluation loss 0.05512853932380676 / accuracy 0.362
(launch_and_fit pid=778) [Client 7] fit, config: {}
(launch_and_fit pid=777) [Client 5] fit, config: {}
(launch_and_fit pid=778) Epoch 1: train loss 0.023649688810110092, accuracy 0.38555555555555554
(launch_and_fit pid=778) [Client 0] fit, config: {}
(launch_and_fit pid=777) Epoch 1: train loss 0.0242004431784153, accuracy 0.37577777777777777


DEBUG flower 2022-06-23 14:19:38,850 | server.py:281 | fit_round received 3 results and 0 failures
INFO flower 2022-06-23 14:19:39,061 | server.py:164 | fit progress: (3, 0.053611843585968016, {'accuracy': 0.396}, 35.78112193399997)
INFO flower 2022-06-23 14:19:39,064 | server.py:209 | evaluate_round: no clients selected, cancel
INFO flower 2022-06-23 14:19:39,068 | server.py:182 | FL finished in 35.78733605099998
INFO flower 2022-06-23 14:19:39,071 | app.py:149 | app_fit: losses_distributed []
INFO flower 2022-06-23 14:19:39,074 | app.py:150 | app_fit: metrics_distributed {}
INFO flower 2022-06-23 14:19:39,075 | app.py:151 | app_fit: losses_centralized [(0, 0.07375648212432862), (1, 0.06275381827354432), (2, 0.05512853932380676), (3, 0.053611843585968016)]
INFO flower 2022-06-23 14:19:39,077 | app.py:152 | app_fit: metrics_centralized {'accuracy': [(0, 0.092), (1, 0.304), (2, 0.362), (3, 0.396)]}


(launch_and_fit pid=778) Epoch 1: train loss 0.02394915744662285, accuracy 0.38422222222222224
Server-side evaluation loss 0.053611843585968016 / accuracy 0.396


History (loss, centralized):
	round 0: 0.07375648212432862
	round 1: 0.06275381827354432
	round 2: 0.05512853932380676
	round 3: 0.053611843585968016
History (metrics, centralized):
{'accuracy': [(0, 0.092), (1, 0.304), (2, 0.362), (3, 0.396)]}

In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        # Read values from config
        current_round = config["current_round"]
        local_epochs = config["local_epochs"]

        # Use values provided by the config
        print(f"[Client {self.cid}, round {current_round}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=local_epochs)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

In [ ]:
def fit_config(rnd: int):
    """Return training configuration dict for each round.
    
    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "current_round": rnd,  # The current round of federated learning
        "local_epochs": 1 if rnd < 2 else 2,  # 
    }
    return config

In [ ]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_eval=0.3,
    min_fit_clients=3,
    min_eval_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.weights_to_parameters(get_parameters(Net())),
    eval_fn=evaluate,
    on_fit_config_fn=fit_config,  # Pass the fit_config function
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    num_rounds=3,  # Just three rounds
    strategy=strategy,
)

INFO flower 2022-06-23 14:19:45,439 | app.py:147 | Ray initialized with resources: {'memory': 7889736500.0, 'object_store_memory': 3944868249.0, 'CPU': 2.0, 'node:172.28.0.2': 1.0}
INFO flower 2022-06-23 14:19:45,447 | app.py:156 | Starting Flower simulation running: {'num_rounds': 3}
INFO flower 2022-06-23 14:19:45,451 | server.py:128 | Initializing global parameters
INFO flower 2022-06-23 14:19:45,455 | server.py:323 | Using initial parameters provided by strategy
INFO flower 2022-06-23 14:19:45,460 | server.py:130 | Evaluating initial parameters
INFO flower 2022-06-23 14:19:45,709 | server.py:137 | initial parameters (loss, other metrics): 0.0738621392250061, {'accuracy': 0.084}
INFO flower 2022-06-23 14:19:45,715 | server.py:143 | FL starting
DEBUG flower 2022-06-23 14:19:45,716 | server.py:269 | fit_round: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.0738621392250061 / accuracy 0.084
(launch_and_fit pid=1020) [Client 3, round 1] fit, config: {'current_round': 1, 'local_epochs': 1}
(launch_and_fit pid=1021) [Client 1, round 1] fit, config: {'current_round': 1, 'local_epochs': 1}
(launch_and_fit pid=1020) Epoch 1: train loss 0.029214102774858475, accuracy 0.2262222222222222
(launch_and_fit pid=1021) Epoch 1: train loss 0.02891010046005249, accuracy 0.24244444444444443
(launch_and_fit pid=1020) [Client 2, round 1] fit, config: {'current_round': 1, 'local_epochs': 1}


DEBUG flower 2022-06-23 14:19:58,243 | server.py:281 | fit_round received 3 results and 0 failures
INFO flower 2022-06-23 14:19:58,444 | server.py:164 | fit progress: (1, 0.06240503478050232, {'accuracy': 0.302}, 12.72770052200002)
INFO flower 2022-06-23 14:19:58,446 | server.py:209 | evaluate_round: no clients selected, cancel
DEBUG flower 2022-06-23 14:19:58,451 | server.py:269 | fit_round: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=1020) Epoch 1: train loss 0.029160788282752037, accuracy 0.2157777777777778
Server-side evaluation loss 0.06240503478050232 / accuracy 0.302
(launch_and_fit pid=1020) [Client 4, round 2] fit, config: {'current_round': 2, 'local_epochs': 2}
(launch_and_fit pid=1021) [Client 6, round 2] fit, config: {'current_round': 2, 'local_epochs': 2}
(launch_and_fit pid=1020) Epoch 1: train loss 0.025801805779337883, accuracy 0.32622222222222225
(launch_and_fit pid=1021) Epoch 1: train loss 0.025917483493685722, accuracy 0.31444444444444447
(launch_and_fit pid=1020) Epoch 2: train loss 0.023479612544178963, accuracy 0.38622222222222224
(launch_and_fit pid=1021) Epoch 2: train loss 0.02384360134601593, accuracy 0.382
(launch_and_fit pid=1020) [Client 0, round 2] fit, config: {'current_round': 2, 'local_epochs': 2}
(launch_and_fit pid=1020) Epoch 1: train loss 0.025814959779381752, accuracy 0.32822222222222225


DEBUG flower 2022-06-23 14:20:16,261 | server.py:281 | fit_round received 3 results and 0 failures
INFO flower 2022-06-23 14:20:16,462 | server.py:164 | fit progress: (2, 0.051755956411361694, {'accuracy': 0.398}, 30.746024709999972)
INFO flower 2022-06-23 14:20:16,464 | server.py:209 | evaluate_round: no clients selected, cancel
DEBUG flower 2022-06-23 14:20:16,468 | server.py:269 | fit_round: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=1020) Epoch 2: train loss 0.023618806153535843, accuracy 0.38866666666666666
Server-side evaluation loss 0.051755956411361694 / accuracy 0.398
(launch_and_fit pid=1020) [Client 8, round 3] fit, config: {'current_round': 3, 'local_epochs': 2}
(launch_and_fit pid=1021) [Client 0, round 3] fit, config: {'current_round': 3, 'local_epochs': 2}
(launch_and_fit pid=1021) Epoch 1: train loss 0.022692853584885597, accuracy 0.406
(launch_and_fit pid=1020) Epoch 1: train loss 0.022672632709145546, accuracy 0.4131111111111111
(launch_and_fit pid=1021) Epoch 2: train loss 0.021439742296934128, accuracy 0.44355555555555554
(launch_and_fit pid=1021) [Client 4, round 3] fit, config: {'current_round': 3, 'local_epochs': 2}
(launch_and_fit pid=1020) Epoch 2: train loss 0.021295150741934776, accuracy 0.44955555555555554
(launch_and_fit pid=1021) Epoch 1: train loss 0.022391710430383682, accuracy 0.42155555555555557


DEBUG flower 2022-06-23 14:20:34,323 | server.py:281 | fit_round received 3 results and 0 failures
INFO flower 2022-06-23 14:20:34,522 | server.py:164 | fit progress: (3, 0.04895351314544678, {'accuracy': 0.444}, 48.80559737599998)
INFO flower 2022-06-23 14:20:34,528 | server.py:209 | evaluate_round: no clients selected, cancel
INFO flower 2022-06-23 14:20:34,529 | server.py:182 | FL finished in 48.812695958999996
INFO flower 2022-06-23 14:20:34,536 | app.py:149 | app_fit: losses_distributed []
INFO flower 2022-06-23 14:20:34,539 | app.py:150 | app_fit: metrics_distributed {}
INFO flower 2022-06-23 14:20:34,540 | app.py:151 | app_fit: losses_centralized [(0, 0.0738621392250061), (1, 0.06240503478050232), (2, 0.051755956411361694), (3, 0.04895351314544678)]
INFO flower 2022-06-23 14:20:34,542 | app.py:152 | app_fit: metrics_centralized {'accuracy': [(0, 0.084), (1, 0.302), (2, 0.398), (3, 0.444)]}


(launch_and_fit pid=1021) Epoch 2: train loss 0.021189486607909203, accuracy 0.45711111111111113
Server-side evaluation loss 0.04895351314544678 / accuracy 0.444


History (loss, centralized):
	round 0: 0.0738621392250061
	round 1: 0.06240503478050232
	round 2: 0.051755956411361694
	round 3: 0.04895351314544678
History (metrics, centralized):
{'accuracy': [(0, 0.084), (1, 0.302), (2, 0.398), (3, 0.444)]}

Scaling federated learning

we can use Flower to experiment with a large number of clients.

In [ ]:
NUM_CLIENTS = 1000

trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
def fit_config(rnd: int):
    config = {
        "current_round": rnd,
        "local_epochs": 3,
    }
    return config

strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.025,  # Train on 25 clients (each round)
    fraction_eval=0.05,  # Evaluate on 50 clients (each round)
    min_fit_clients=20,
    min_eval_clients=40,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.weights_to_parameters(get_parameters(Net())),
    on_fit_config_fn=fit_config,
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    num_rounds=3,
    strategy=strategy,
)

INFO flower 2022-06-23 14:20:43,106 | app.py:147 | Ray initialized with resources: {'CPU': 2.0, 'object_store_memory': 3944944435.0, 'node:172.28.0.2': 1.0, 'memory': 7889888871.0}
INFO flower 2022-06-23 14:20:43,110 | app.py:156 | Starting Flower simulation running: {'num_rounds': 3}
INFO flower 2022-06-23 14:20:43,135 | server.py:128 | Initializing global parameters
INFO flower 2022-06-23 14:20:43,143 | server.py:323 | Using initial parameters provided by strategy
INFO flower 2022-06-23 14:20:43,152 | server.py:130 | Evaluating initial parameters
INFO flower 2022-06-23 14:20:43,155 | server.py:143 | FL starting
DEBUG flower 2022-06-23 14:20:43,162 | server.py:269 | fit_round: strategy sampled 25 clients (out of 1000)


(launch_and_fit pid=1261) [Client 184, round 1] fit, config: {'current_round': 1, 'local_epochs': 3}
(launch_and_fit pid=1262) [Client 956, round 1] fit, config: {'current_round': 1, 'local_epochs': 3}
(launch_and_fit pid=1262) Epoch 1: train loss 0.0004614427452906966, accuracy 0.2
(launch_and_fit pid=1262) Epoch 2: train loss 0.000457173737231642, accuracy 0.15555555555555556
(launch_and_fit pid=1261) Epoch 1: train loss 0.00046183165977708995, accuracy 0.06666666666666667
(launch_and_fit pid=1261) Epoch 2: train loss 0.0004552064056042582, accuracy 0.1111111111111111
(launch_and_fit pid=1262) Epoch 3: train loss 0.000456655106972903, accuracy 0.15555555555555556
(launch_and_fit pid=1261) Epoch 3: train loss 0.00045476044761016965, accuracy 0.26666666666666666
(launch_and_fit pid=1262) [Client 242, round 1] fit, config: {'current_round': 1, 'local_epochs': 3}
(launch_and_fit pid=1262) Epoch 1: train loss 0.0004611019103322178, accuracy 0.06666666666666667
(launch_and_fit pid=1262) Ep

(raylet) Spilled 3359 MiB, 36 objects, write throughput 193 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.


(launch_and_fit pid=1262) [Client 475, round 1] fit, config: {'current_round': 1, 'local_epochs': 3}
(launch_and_fit pid=1262) Epoch 1: train loss 0.0004597997176460922, accuracy 0.13333333333333333
(launch_and_fit pid=1261) [Client 394, round 1] fit, config: {'current_round': 1, 'local_epochs': 3}
(launch_and_fit pid=1262) 
(launch_and_fit pid=1262) Epoch 2: train loss 0.0004569399752654135, accuracy 0.13333333333333333
(launch_and_fit pid=1261) Epoch 1: train loss 0.00046248387661762536, accuracy 0.06666666666666667
(launch_and_fit pid=1262) Epoch 3: train loss 0.000449405109975487, accuracy 0.13333333333333333
(launch_and_fit pid=1261) Epoch 2: train loss 0.0004565802519209683, accuracy 0.08888888888888889
(launch_and_fit pid=1261) Epoch 3: train loss 0.000454375782283023, accuracy 0.08888888888888889
(launch_and_fit pid=1261) [Client 468, round 1] fit, config: {'current_round': 1, 'local_epochs': 3}
(launch_and_fit pid=1262) [Client 225, round 1] fit, config: {'current_round': 1, '

DEBUG flower 2022-06-23 14:21:42,169 | server.py:281 | fit_round received 25 results and 0 failures


...

DEBUG flower 2022-06-23 14:26:44,620 | server.py:215 | evaluate_round: strategy sampled 50 clients (out of 1000)


(launch_and_evaluate pid=1262) [Client 395] evaluate, config: {}
(launch_and_evaluate pid=1261) [Client 929] evaluate, config: {}
(launch_and_evaluate pid=1262) [Client 277] evaluate, config: {}
(launch_and_evaluate pid=1261) [Client 642] evaluate, config: {}
(launch_and_evaluate pid=1262) [Client 848] evaluate, config: {}
(launch_and_evaluate pid=1261) [Client 784] evaluate, config: {}
(launch_and_evaluate pid=1262) [Client 180] evaluate, config: {}
(launch_and_evaluate pid=1261) [Client 507] evaluate, config: {}
(launch_and_evaluate pid=1261) [Client 786] evaluate, config: {}
(launch_and_evaluate pid=1261) [Client 991] evaluate, config: {}
(launch_and_evaluate pid=1261) [Client 129] evaluate, config: {}
(launch_and_evaluate pid=1262) [Client 391] evaluate, config: {}
(launch_and_evaluate pid=1261) [Client 785] evaluate, config: {}
(launch_and_evaluate pid=1262) [Client 387] evaluate, config: {}
(launch_and_evaluate pid=1262) [Client 349] evaluate, config: {}
(launch_and_evaluate pid=

DEBUG flower 2022-06-23 14:28:23,573 | server.py:227 | evaluate_round received 50 results and 0 failures
INFO flower 2022-06-23 14:28:23,575 | server.py:182 | FL finished in 460.41334921699996
INFO flower 2022-06-23 14:28:23,591 | app.py:149 | app_fit: losses_distributed [(1, 0.4581334037780763), (2, 0.45518056106567395), (3, 0.44773124217987054)]
INFO flower 2022-06-23 14:28:23,592 | app.py:150 | app_fit: metrics_distributed {}
INFO flower 2022-06-23 14:28:23,598 | app.py:151 | app_fit: losses_centralized []
INFO flower 2022-06-23 14:28:23,600 | app.py:152 | app_fit: metrics_centralized {}


(launch_and_evaluate pid=1262) [Client 360] evaluate, config: {}
(launch_and_evaluate pid=1261) [Client 677] evaluate, config: {}
(launch_and_evaluate pid=1262) [Client 139] evaluate, config: {}
(launch_and_evaluate pid=1261) [Client 964] evaluate, config: {}


History (loss, distributed):
	round 1: 0.4581334037780763
	round 2: 0.45518056106567395
	round 3: 0.44773124217987054